In [29]:
import os
import boto3
import pandas as pd

In [30]:
#create an s3 client
AWS_SERVER_PUBLIC_KEY = os.environ.get('AWS_ACCESS_KEY_ID') 
AWS_SERVER_SECRET_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY') 

s3_client = boto3.client('s3', 
                         endpoint_url='https://nrs.objectstore.gov.bc.ca',
                         aws_access_key_id=AWS_SERVER_PUBLIC_KEY,
                         aws_secret_access_key=AWS_SERVER_SECRET_KEY)

In [31]:
#list buckets
response = s3_client.list_buckets()
for bucket in response['Buckets']:
    print(bucket['Name'])

pmecwy


In [32]:
# list objects in the 'pmecwy' bucket
bucket_name = 'pmecwy'
response = s3_client.list_objects_v2(Bucket=bucket_name)

##show only 10 first objects
for obj in response.get('Contents', [])[:10]:
    print(obj['Key'])

CEA_Images/
CEA_Images/Visuals/
CEA_Images/Visuals/2018-08-27 12.48.45 Sugar Lake.jpg
CEA_Images/Visuals/Bowron_original.jpg
CEA_Images/Visuals/IMG_0171.JPG
CEA_Images/Visuals/IMG_0176.JPG
CEA_Images/Visuals/IMG_0178.JPG
CEA_Images/Visuals/IMG_1545.JPG
CEA_Images/Visuals/IMG_1576.JPG
CEA_Images/Visuals/IMG_1576_crop.jpg


In [33]:
#upload an xlsx to the bucket
subfolder_name = 'test_ml'
file_path= r'W:\lwbc\visr\Workarea\moez_labiadh\LEARNING\object_storage\thompsonWSHD_oldForest_stats.xlsx'
file_name= os.path.basename(file_path)

s3_client.upload_file(file_path, bucket_name, f'{subfolder_name}/{file_name}')

print(f"File {file_name}' uploaded successfully to subfolder '{subfolder_name}'.")

File thompsonWSHD_oldForest_stats.xlsx' uploaded successfully to subfolder 'test_ml'.


In [36]:
# read the xlsx into df
s3_object_key = f'{subfolder_name}/{file_name}'
obj = s3_client.get_object(Bucket=bucket_name, Key=s3_object_key)
df = pd.read_excel(obj['Body'].read())
df.head()

,WATERSHED_GROUP_NAME,CATEGORY,BEC_ZONE,AREA_HA
0,Adams River,ANCIENT & REMNANT,ICH,79.62
1,Adams River,ANCIENT FOREST,ESSF,97.70
2,Adams River,ANCIENT FOREST,ICH,198.43
3,Adams River,OGMA,ESSF,15181.92
4,Adams River,OGMA,ICH,16144.44
5,Adams River,OGMA,IDF,2990.64
6,Adams River,OGMA,MS,458.84
7,Adams River,OLD FOREST,ESSF,2099.61
8,Adams River,OLD FOREST,ICH,5824.94
9,Adams River,OLD FOREST,IDF,155.09


In [37]:
#write data to the xlsx folder
new_row = {'WATERSHED_GROUP_NAME': 'XXX',
           'CATEGORY': 'YYY',
           'BEC_ZONE': 'ZZZ',
           'AREA_HA': 20.15}
df.loc[0] = new_row
df.reset_index(drop=True, inplace=True)
df.head()

,WATERSHED_GROUP_NAME,CATEGORY,BEC_ZONE,AREA_HA
0,XXX,YYY,ZZZ,20.15
1,Adams River,ANCIENT FOREST,ESSF,97.70
2,Adams River,ANCIENT FOREST,ICH,198.43
3,Adams River,OGMA,ESSF,15181.92
4,Adams River,OGMA,ICH,16144.44
